<a href="https://colab.research.google.com/github/Entspannter/CrispCerebella/blob/main/DL_Test_3_Models_HP_K_Fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Entspannter/CrispCerebella.git

Cloning into 'CrispCerebella'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 160 (delta 56), reused 126 (delta 32), pack-reused 0
Receiving objects: 100% (160/160), 75.57 MiB | 13.70 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (145/145), done.


In [2]:
pip install -q -U keras-tuner

     |████████████████████████████████| 102kB 6.5MB/s 


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import os 
from pandas import read_csv
from pandas import DataFrame
import numpy as np
from numpy import mean
from numpy import std
from numpy import array
from numpy import vstack
from numpy import dstack
from numpy import unique
from scipy import stats
from matplotlib import pyplot
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import keras_tuner
from keras_tuner import HyperModel
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Functions 

In [4]:
#Build a Dataloader function

def dataloader(d_path):
	dataframe = read_csv(d_path, header=None, delim_whitespace=True)
	return dataframe.values

data = dataloader('/content/CrispCerebella/Dataset/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt')

print("Shape of Total_Acc_y_train:", data.shape)

# function to load a group of data and create a 3 dimensional tensor out of it
def dataloader_group(all_files, directory=''):
	datalist = list()
 # for each element in the filelist we load each file seperately and append its data to a list
	for elem in all_files:
		data = dataloader(directory + elem)
		datalist.append(data)
	# create a 3 dimensional tensor out of the individual data files
	datalist = dstack(datalist)
	return datalist

# load all acc data into a 3 dimensional tensor 
total_acc_xyz = dataloader_group(['total_acc_x_train.txt', 'total_acc_y_train.txt', 'total_acc_z_train.txt'], directory='/content/CrispCerebella/Dataset/UCI HAR Dataset/train/Inertial Signals/')
print("3D Form of Total Acc Data",total_acc_xyz.shape)

# create a train and test group out of the data
def datasetloader(train_test_var, directory=''):

  
	filepath = directory + train_test_var + '/Inertial Signals/'
	filenames = sorted(os.listdir(filepath))

	# load X data (input)
	X = dataloader_group(filenames, filepath)
	# load output labels
	y = dataloader(directory + train_test_var + '/y_'+train_test_var+'.txt')
	return X, y

Shape of Total_Acc_y_train: (7352, 128)
3D Form of Total Acc Data (7352, 128, 3)


In [5]:
# load the dataset, returns train and test X and y elements
def transform_dataset(trainX, trainy, testX, testy):
	print(trainX.shape, trainy.shape)
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [6]:
def HP_Search(hypermodel ,MaxTrial, Num_Epochs_Search, Train_X, Train_y, ID):
# Define the tuner 
  tuner = keras_tuner.BayesianOptimization(
    hypermodel,
    objective = 'val_accuracy',
    max_trials = MaxTrial,
    tuner_id= ID,
    overwrite = True
)
  # Define an early stop 
  es = EarlyStopping(monitor='val_accuracy', mode='max', verbose = 2, patience = 3)

  # Start the search og hyperparemeters
  tuner.search(Train_X, Train_y, epochs = Num_Epochs_Search, validation_split = 0.2,callbacks=[es])

  # Get the tuned hyperparemters 
  best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
  print("[The tuned hyperparemters]:", best_hps.get_config()['values'])

  # build a new model with the best hyperparemters
  model = tuner.hypermodel.build(best_hps)

  return model, best_hps

In [7]:
def K_Fold_CV (X, y, Model_Class, Model_Name, Input_Shape, Num_Outputs, Max_Trials, Num_Epochs_Search, Num_Epochs_Fit, Num_K_Folds):

  kf = KFold(n_splits = Num_K_Folds, random_state=None)
  acc_score = []
  k = 0
  for train_index , test_index in kf.split(X):
      print("Statring fold number {}". format(k+1))
      X_train , X_test = X[train_index,:],X[test_index,:]
      y_train , y_test = y[train_index] , y[test_index]

      hypermodel = Model_Class(Input_Shape, Num_Outputs)

      Best_Model, best_hp = HP_Search(hypermodel, Max_Trials, Num_Epochs_Search, X_train, y_train, "Tuner_"+str(Model_Name))

      Best_Model.fit(X_train, y_train, epochs = Num_Epochs_Fit, validation_split = 0.2)

      eval_result = Best_Model.evaluate(X_test, y_test)

      test_acc = eval_result[1]

      acc_score.append(test_acc)

      if k != 0:
        if test_acc > acc_score[k-1]:
          print("{} > {} ". format(test_acc,acc_score[k-1]))
          Best_HP = best_hp
          print("Overriding the HP")

      k += 1
      print("Fold number {} is done". format(k+1))



  avg_acc_score = sum(acc_score)/Num_K_Folds

  print('Accuracy of each fold - {}'.format(acc_score))
  print('Avg accuracy : {}'.format(avg_acc_score))

  return acc_score, avg_acc_score, Best_HP

In [8]:
def evaluate_model(trainX, trainy, testX, testy, Input_Shape, Num_Outputs, Best_HP, Model_Class, Num_Epochs_Fit, Batch_Size):

	# create a new instace of the model class
	model = Model_Class(Input_Shape, Num_Outputs)

	# Build a new model with the best hyperparameters
	Best_Model = model.build(Best_HP)
 
	# Fit the model 
	Best_Model.fit(trainX, trainy, epochs = Num_Epochs_Fit, validation_split = 0.2)
 
	# evaluate model
	_, accuracy = Best_Model.evaluate(testX, testy, batch_size = Batch_Size, verbose=0)
 
	print("The accureay of the model is {}". format(accuracy))
 
	return Best_Model, accuracy

# Stacked LSTM - 2 Layers

In [9]:
# load all train
trainX, trainy = datasetloader('train', '/content/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of training Data:",trainX.shape, trainy.shape)
# load all test
testX, testy = datasetloader('test', '/content/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of test Data:",testX.shape, testy.shape)

Shape of training Data: (7352, 128, 9) (7352, 1)
Shape of test Data: (2947, 128, 9) (2947, 1)


In [10]:
class LSTMStacked(HyperModel):
    def __init__(self, input_shape,n_outputs):
        self.input_shape = input_shape
        self.n_outputs = n_outputs

    def build(self, hp):

        model = keras.Sequential()

        model.add(LSTM(
                        units=hp.Int(
                            'Units_LSTM_1',
                            min_value=32,
                            max_value=512,
                            step=32,
                            default=128
                        ),
                       input_shape=self.input_shape,
                       return_sequences = True
                    )
                )
        

        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
                    )
                  )
                )
        


        model.add(LSTM(
                        units=hp.Int(
                            'Units_LSTM_2',
                            min_value=32,
                            max_value=512,
                            step=32,
                            default=128
                        )
                    )
                )
        

        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
                    )
                  )
                )
        


        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=64,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )

        model.add(Dense(self.n_outputs, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model

In [11]:
trainX, trainy, testX, testy = transform_dataset(trainX, trainy, testX, testy)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [12]:
n_timesteps, n_features, N_OUTPUTS = trainX.shape[1], trainX.shape[2], trainy.shape[1]

n_steps, n_length = 4, 32

In [13]:
INPUT_SHAPE=(n_timesteps,n_features)

In [14]:
Num_Epochs_Search = 40
Num_Epochs_Fit = 40
Max_Trials = 10
Num_K_Folds = 5
Model_Name_LSTM = "LSTM"
batch_size = 64

In [15]:
X = np.concatenate((trainX, testX),axis=0)

y = np.concatenate((trainy, testy),axis=0)

In [ ]:
acc_score_LSTM , avg_acc_score_LSTM, Best_HP_LSTM = K_Fold_CV(X, 
                                             y, 
                                             LSTMStacked, 
                                             Model_Name_LSTM, 
                                             INPUT_SHAPE,
                                             N_OUTPUTS,
                                             Max_Trials,
                                             Num_Epochs_Search,
                                             Num_Epochs_Fit, 
                                             Num_K_Folds)

Statring fold number 1

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
Units_LSTM_1      |416               |?                 
dropout_1         |0.15              |?                 
Units_LSTM_2      |512               |?                 
dropout_2         |0.35              |?                 
units             |128               |?                 
dense_activation  |relu              |?                 
learning_rate     |0.00010174        |?                 

Epoch 1/40
  6/206 [..............................] - ETA: 25s - loss: 1.7728 - accuracy: 0.2865WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0562s vs `on_train_batch_end` time: 0.0693s). Check your callbacks.


In [ ]:
Best_HP_LSTM.get_config()['values']

In [ ]:
# Build a new model with the HP, train it with the inital training set and eveluate it with the test set 

BestModelLSTM, LSTM_accuracy = evaluate_model(trainX,
                               trainy,
                               testX,
                               testy,
                               INPUT_SHAPE,
                               N_OUTPUTS,
                               Best_HP_LSTM,
                               LSTMStacked,
                               Num_Epochs_Fit,
                               batch_size)

In [ ]:
Best_Model_LSTM.save("/content/CrispCerebella/Models/Final_Model_LSTM")

### For Later Predictions:
---



In [ ]:
''''from sklearn import metrics
from sklearn.metrics import classification_report
prediction = np.argmax(Best_Model_LSTM.predict(testX), axis=-1)
print(prediction)

prediction = prediction.reshape(prediction.shape[0],1)
prediction = prediction+1

report = classification_report(testy, prediction)
print(report)''''

#ConvLSTM

In [ ]:
trainX, trainy = datasetloader('train', '/content/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of training Data:",trainX.shape, trainy.shape)
# load all test
testX, testy = datasetloader('test', '/content/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of test Data:",testX.shape, testy.shape)

In [ ]:
class ConvLSTM(HyperModel):
    def __init__(self, input_shape,n_outputs):
        self.input_shape = input_shape
        self.n_outputs = n_outputs

    def build(self, hp):


        model = keras.Sequential()

        model.add(
            ConvLSTM2D(
                filters=hp.Choice(
                        'num_filters',
                        values=[16, 32, 64],
                        default=64,
                ),
                kernel_size=(1,3),
                activation='relu',
                input_shape=self.input_shape
            )
        )

        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        model.add(Flatten())
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_2',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        model.add(Dense(self.n_outputs, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

In [ ]:
trainX, trainy, testX, testy = transform_dataset(trainX, trainy, testX, testy)

In [ ]:
n_timesteps, n_features, N_OUTPUTS = trainX.shape[1], trainX.shape[2], trainy.shape[1]

n_steps, n_length = 4, 32

In [ ]:
INPUT_SHAPE = (n_steps, 1, n_length, n_features)

In [ ]:
# reshape into subsequences (samples, time steps, rows, cols, channels)
trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))

In [ ]:
Num_Epochs_Fit = 10
Num_K_Folds = 5
Model_Name_Conv = "ConvLSTM"

In [ ]:
X_ConvLSTM = np.concatenate((trainX, testX),axis=0)

y_ConvLSTM = np.concatenate((trainy, testy),axis=0)

In [ ]:
acc_score_ConvLSTM , avg_acc_score_ConvLSTM, Best_HP_ConvLSTM = K_Fold_CV(X, 
                                                                          y, 
                                                                          ConvLSTM,
                                                                          Model_Name_Conv,
                                                                          INPUT_SHAPE,
                                                                          N_OUTPUTS,
                                                                          Max_Trials,
                                                                          Num_Epochs_Search,
                                                                          Num_Epochs_Fit,
                                                                          Num_K_Folds)

In [ ]:
BestModelConvLSTM, ConvLSTM_accuracy = evaluate_model(trainX,
                                                      trainy,
                                                      testX,
                                                      testy,
                                                      INPUT_SHAPE,
                                                      N_OUTPUTS,
                                                      Best_HP_ConvLSTM,
                                                      ConvLSTM,
                                                      Num_Epochs_Fit,
                                                      batch_size)

In [ ]:
BestModelConvLSTM.save("/content/CrispCerebella/Models/Final_Model_ConvLSTM")


# CNN-LSTM
 

In [ ]:
  # load all train
trainX, trainy = datasetloader('train', '/content/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of training Data:",trainX.shape, trainy.shape)
# load all test
testX, testy = datasetloader('test', '/content/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of test Data:",testX.shape, testy.shape)

In [ ]:
class CNNLSTM(HyperModel):
    def __init__(self, input_shape,n_outputs):
        self.input_shape = input_shape
        self.n_outputs = n_outputs

    def build(self, hp):

        model = keras.Sequential()
        model.add(TimeDistributed(
            Conv1D(
                filters=hp.Choice(
                        'num_filters_1',
                        values=[16, 32, 64],
                        default=64,
                ),
                kernel_size=3,
                activation='relu',
                input_shape=self.input_shape,
            )
          )   
        )
        model.add(TimeDistributed(
            Conv1D(
                filters=hp.Choice(
                        'num_filters_2',
                        values=[16, 32, 64],
                        default=64,
                ),
                kernel_size=3,
                activation='relu'
            )
          )   
        )

        model.add(TimeDistributed(Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )
           )
          )
        )
      
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

        model.add(TimeDistributed(Flatten()))

        model.add(
            LSTM(
                units=hp.Int(
                    'units_LSTM',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                )
            )
        )

        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_2',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )

        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(Dense(self.n_outputs, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

In [ ]:
trainX, trainy, testX, testy = transform_dataset(trainX, trainy, testX, testy)

In [ ]:
	n_timesteps, n_features, N_OUTPUTS = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 4, 32

In [ ]:
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))

In [ ]:
INPUT_SHAPE = (None, n_length,n_features)

In [ ]:
Num_Epochs_Fit = 10
Num_K_Folds = 5
Model_Name_CNN = "CNNLSTM"

In [ ]:
X_CNNLSTM = np.concatenate((trainX, testX),axis=0)

y_CNNLSTM = np.concatenate((trainy, testy),axis=0)

In [ ]:
acc_score_CNNLSTM , avg_acc_score_CNNLSTM,Best_HP_CNNLSTM = K_Fold_CV(X,
                                                                      y,
                                                                      CNNLSTM,
                                                                      Model_Name_CNN,
                                                                      INPUT_SHAPE,
                                                                      N_OUTPUTS,
                                                                      Max_Trials,
                                                                      Num_Epochs_Search,
                                                                      Num_Epochs_Fit,
                                                                      Num_K_Folds)

In [ ]:
BestModelCNNLSTM, CNNLSTM_accuracy = evaluate_model(trainX,
                                                      trainy,
                                                      testX,
                                                      testy,
                                                      INPUT_SHAPE,
                                                      N_OUTPUTS,
                                                      Best_HP_CNNLSTM,
                                                      CNNLSTM,
                                                      Num_Epochs_Fit,
                                                      batch_size)

In [ ]:
BestModelCNNLSTM.save("/content/CrispCerebella/Models/Final_Model_CNNLSTM")